In [ ]:
# storing and retrieving data
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.integrate import solve_ivp
from scipy import integrate
from matplotlib.pyplot import figure
from itertools import repeat
import seaborn as sn
import ordpy
import pandas as pd
import array

alpha = 0
alpha_array_inner_layer = []
alpha_array_outer_layer = []
beta_array = []
PD_x1_list = []
PD_y1_list = []
PD_x2_list = []
PD_y2_list = []
beta = 0
alpha_max = 2.0
beta_max = 0.4
beta_repeat_times = int((beta_max-beta)/0.005)

while beta <= beta_max:
    while alpha <= alpha_max:
        alpha_array_inner_layer.append(alpha)
        alpha += 0.025
    beta_array.extend(repeat(beta,beta_repeat_times))
    alpha_array_outer_layer.extend(alpha_array_inner_layer)
    beta += 0.005
    
print(len(beta_array), len(alpha_array_outer_layer))
    
xi_0 = 0.175
T_1_ = 5
xi_P = 0.175
A_P = 0.01
omega_P = 0.7
A_0 = 0.5
omega_T_ = 0.7
c = 0.1
b_0 = 0.6
a = 0.7
T_2_ = 5
g = 0.1

def JJ (t, y, xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g):
    x1, y1, x2, y2, z = y
    dx1 = x1*(1-xi_0*math.e**(-1/T_1_)-xi_P)-y1-1/3*x1**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_1_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy1 = c*(-b_0*math.e**(1/T_1_)*y1+a+x1)
    dx2 = x2*(1-xi_0*math.e**(-1/T_2_)-xi_P)-y2-1/3*x2**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_2_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy2 = c*(-b_0*math.e**(1/T_2_)*y2+a+x2)
    dz = g*(x1-x2)
    dydt = [dx1, dy1, dx2, dy2, dz]
    return dydt

tStart = 0
tEnd = 8000

X0 = [0.2, 0.1, 0.02, 0.01, 0.001]

k = 0
while k < len(beta_array) and k < len(alpha_array_outer_layer):
    alpha = alpha_array_outer_layer[k]
    beta = beta_array[k]
    
    solution = integrate.solve_ivp(JJ, [tStart, tEnd], X0, method='RK45', t_eval=np.linspace(tStart,tEnd,100000), args=(xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g))
    
    t_sol = solution.t
    x1_sol = solution.y[0]
    y1_sol = solution.y[1]
    x2_sol = solution.y[2]
    y2_sol = solution.y[3]
    z_sol = solution.y[4]
    
    # P(pattern0123)-P(pattern3210) of x1
    # list of probabilities of ordinal distributions
    OD_x1 = ordpy.ordinal_distribution(x1_sol, dx=4, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None)
    # probability difference P(pattern0123)-P(pattern3210)
    PD_x1 = OD_x1[1][0]-OD_x1[1][len(OD_x1[1])-1]
    PD_x1_list.append(PD_x1)
    
    # P(pattern0123)-P(pattern3210) of y1
    # list of probabilities of ordinal distributions
    OD_y1 = ordpy.ordinal_distribution(y1_sol, dx=4, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None)
    # probability difference P(pattern0123)-P(pattern3210)
    PD_y1 = OD_y1[1][0]-OD_y1[1][len(OD_y1[1])-1]
    PD_y1_list.append(PD_y1)
    
    # P(pattern0123)-P(pattern3210) of x2
    # list of probabilities of ordinal distributions
    OD_x2 = ordpy.ordinal_distribution(x2_sol, dx=4, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None)
    # probability difference P(pattern0123)-P(pattern3210)
    PD_x2 = OD_x2[1][0]-OD_x2[1][len(OD_x2[1])-1]
    PD_x2_list.append(PD_x2)
    
    # P(pattern0123)-P(pattern3210) of y2
    # list of probabilities of ordinal distributions
    OD_y2 = ordpy.ordinal_distribution(y2_sol, dx=4, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None)
    # probability difference P(pattern0123)-P(pattern3210)
    PD_y2 = OD_y2[1][0]-OD_y2[1][len(OD_y2[1])-1]
    PD_y2_list.append(PD_y2)
    
    k += 1

PD_x1_matrix = np.array_split(PD_x1_list, beta_repeat_times)
PD_y1_matrix = np.array_split(PD_y1_list, beta_repeat_times)
PD_x2_matrix = np.array_split(PD_x2_list, beta_repeat_times)
PD_y2_matrix = np.array_split(PD_y2_list, beta_repeat_times)

from numpy import savetxt
savetxt('PD_x1.csv', PD_x1_matrix, delimiter=',')
savetxt('PD_y1.csv', PD_y1_matrix, delimiter=',')
savetxt('PD_x2.csv', PD_x2_matrix, delimiter=',')
savetxt('PD_y2.csv', PD_y2_matrix, delimiter=',')

from numpy import loadtxt
PD_x1_matrix_opened = loadtxt('PD_x1.csv', delimiter=',')
PD_y1_matrix_opened = loadtxt('PD_y1.csv', delimiter=',')
PD_x2_matrix_opened = loadtxt('PD_x2.csv', delimiter=',')
PD_y2_matrix_opened = loadtxt('PD_y2.csv', delimiter=',')

In [ ]:
# heatmap for x1
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_PD_x1 = np.vstack(PD_x1_matrix_opened)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_PD_x1 = sn.heatmap(data=con_PD_x1, annot=True, annot_kws={"size":20})
# alpha tick labels
alpha_ticklabels = 0
list_alpha_ticklabels = []
while alpha_ticklabels <= 1.975:
    list_alpha_ticklabels.append('{:,}'.format(round(alpha_ticklabels, 5)))
    alpha_ticklabels += 0.025
# beta tick labels
beta_ticklabels = 0
list_beta_ticklabels = []
while beta_ticklabels <= 0.4:
    list_beta_ticklabels.append('{:,}'.format(round(beta_ticklabels, 5)))
    beta_ticklabels += 0.005
# heatmap plotting
heatmap_PD_x1.set_xticklabels(list_alpha_ticklabels, fontsize=25)
heatmap_PD_x1.set_yticklabels(list_beta_ticklabels, fontsize=25)
heatmap_PD_x1.invert_yaxis()
heatmap_PD_x1.set_title('Probability Difference between Ordinal Pattern 0123 and 3210 of x1', fontsize=60)
heatmap_PD_x1.set_xlabel('alpha', fontsize=60)
heatmap_PD_x1.set_ylabel('beta', fontsize=60)
cbar = heatmap_PD_x1.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_PD_x1)

In [ ]:
# heatmap for y1
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_PD_y1 = np.vstack(PD_y1_matrix_opened)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_PD_y1 = sn.heatmap(data=con_PD_y1, annot=True, annot_kws={"size":20})
# alpha tick labels
alpha_ticklabels = 0
list_alpha_ticklabels = []
while alpha_ticklabels <= 1.975:
    list_alpha_ticklabels.append('{:,}'.format(round(alpha_ticklabels, 5)))
    alpha_ticklabels += 0.025
# beta tick labels
beta_ticklabels = 0
list_beta_ticklabels = []
while beta_ticklabels <= 0.4:
    list_beta_ticklabels.append('{:,}'.format(round(beta_ticklabels, 5)))
    beta_ticklabels += 0.005
# heatmap plotting
heatmap_PD_y1.set_xticklabels(list_alpha_ticklabels, fontsize=25)
heatmap_PD_y1.set_yticklabels(list_beta_ticklabels, fontsize=25)
heatmap_PD_y1.invert_yaxis()
heatmap_PD_y1.set_title('Probability Difference between Ordinal Pattern 0123 and 3210 of y1', fontsize=60)
heatmap_PD_y1.set_xlabel('alpha', fontsize=60)
heatmap_PD_y1.set_ylabel('beta', fontsize=60)
cbar = heatmap_PD_y1.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_PD_y1)

In [ ]:
# heatmap for x2
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_PD_x2 = np.vstack(PD_x2_matrix_opened)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_PD_x2 = sn.heatmap(data=con_PD_x2, annot=True, annot_kws={"size":20})
# alpha tick labels
alpha_ticklabels = 0
list_alpha_ticklabels = []
while alpha_ticklabels <= 1.975:
    list_alpha_ticklabels.append('{:,}'.format(round(alpha_ticklabels, 5)))
    alpha_ticklabels += 0.025
# beta tick labels
beta_ticklabels = 0
list_beta_ticklabels = []
while beta_ticklabels <= 0.4:
    list_beta_ticklabels.append('{:,}'.format(round(beta_ticklabels, 5)))
    beta_ticklabels += 0.005
# heatmap plotting
heatmap_PD_x2.set_xticklabels(list_alpha_ticklabels, fontsize=25)
heatmap_PD_x2.set_yticklabels(list_beta_ticklabels, fontsize=25)
heatmap_PD_x2.invert_yaxis()
heatmap_PD_x2.set_title('Probability Difference between Ordinal Pattern 0123 and 3210 of x2', fontsize=60)
heatmap_PD_x2.set_xlabel('alpha', fontsize=60)
heatmap_PD_x2.set_ylabel('beta', fontsize=60)
cbar = heatmap_PD_x2.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_PD_x2)

In [ ]:
# heatmap for y2
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

con_PD_y2 = np.vstack(PD_y2_matrix_opened)
sn.set(rc = {'figure.figsize':(150, 150)})
heatmap_PD_y2 = sn.heatmap(data=con_PD_y2, annot=True, annot_kws={"size":20})
# alpha tick labels
alpha_ticklabels = 0
list_alpha_ticklabels = []
while alpha_ticklabels <= 1.975:
    list_alpha_ticklabels.append('{:,}'.format(round(alpha_ticklabels, 5)))
    alpha_ticklabels += 0.025
# beta tick labels
beta_ticklabels = 0
list_beta_ticklabels = []
while beta_ticklabels <= 0.4:
    list_beta_ticklabels.append('{:,}'.format(round(beta_ticklabels, 5)))
    beta_ticklabels += 0.005
# heatmap plotting
heatmap_PD_y2.set_xticklabels(list_alpha_ticklabels, fontsize=25)
heatmap_PD_y2.set_yticklabels(list_beta_ticklabels, fontsize=25)
heatmap_PD_y2.invert_yaxis()
heatmap_PD_y2.set_title('Probability Difference between Ordinal Pattern 0123 and 3210 of y2', fontsize=60)
heatmap_PD_y2.set_xlabel('alpha', fontsize=60)
heatmap_PD_y2.set_ylabel('beta', fontsize=60)
cbar = heatmap_PD_y2.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)
plt.show()
print(heatmap_PD_y2)